<a href="https://colab.research.google.com/github/Trishul32/BIS_LAB/blob/main/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import random

# --- Distance Function ---
def total_distance(route, dist_matrix):
    distance = 0
    for i in range(len(route) - 1):
        distance += dist_matrix[route[i], route[i+1]]
    distance += dist_matrix[route[-1], route[0]]  # return to start
    return distance

# --- Lévy Flight for TSP (random route modification) ---
def levy_flight_route(route):
    new_route = route.copy()
    n = len(route)

    # perform a few random swaps (simulate Lévy flight-like exploration)
    num_swaps = random.randint(1, 4)
    for _ in range(num_swaps):
        i, j = random.sample(range(n), 2)
        new_route[i], new_route[j] = new_route[j], new_route[i]
    return new_route

# --- Cuckoo Search for TSP ---
def cuckoo_search_tsp(dist_matrix, n_nests=20, pa=0.25, max_iter=500):
    num_cities = len(dist_matrix)

    # Initialize nests (routes)
    nests = [random.sample(range(num_cities), num_cities) for _ in range(n_nests)]
    fitness = [total_distance(route, dist_matrix) for route in nests]

    best_route = nests[np.argmin(fitness)]
    best_score = min(fitness)

    for t in range(max_iter):
        for i in range(n_nests):
            # Generate a new solution using Lévy flight
            new_route = levy_flight_route(nests[i])
            new_fitness = total_distance(new_route, dist_matrix)

            # Greedy replacement
            if new_fitness < fitness[i]:
                nests[i] = new_route
                fitness[i] = new_fitness

        # Abandon some nests (with probability pa)
        for i in range(n_nests):
            if random.random() < pa:
                nests[i] = random.sample(range(num_cities), num_cities)
                fitness[i] = total_distance(nests[i], dist_matrix)

        # Update best solution
        current_best_idx = np.argmin(fitness)
        if fitness[current_best_idx] < best_score:
            best_score = fitness[current_best_idx]
            best_route = nests[current_best_idx]

        # Show progress
        if t % 50 == 0:
            print(f"Iteration {t}: Best distance = {best_score:.2f}")

    print("\nFinal Best Route:", best_route)
    print("Final Best Distance:", best_score)
    return best_route, best_score

# --- Example Run ---
if __name__ == "__main__":
    # Example: 8 cities with symmetric distances
    dist_matrix = np.array([
        [0, 12, 10, 19, 8, 7, 15, 20],
        [12, 0, 3, 7, 2, 8, 6, 9],
        [10, 3, 0, 6, 20, 9, 7, 4],
        [19, 7, 6, 0, 12, 11, 10, 8],
        [8, 2, 20, 12, 0, 5, 9, 13],
        [7, 8, 9, 11, 5, 0, 6, 7],
        [15, 6, 7, 10, 9, 6, 0, 5],
        [20, 9, 4, 8, 13, 7, 5, 0]
    ])

    best_route, best_score = cuckoo_search_tsp(dist_matrix)


Iteration 0: Best distance = 50.00
Iteration 50: Best distance = 45.00
Iteration 100: Best distance = 45.00
Iteration 150: Best distance = 45.00
Iteration 200: Best distance = 45.00
Iteration 250: Best distance = 45.00
Iteration 300: Best distance = 45.00
Iteration 350: Best distance = 45.00
Iteration 400: Best distance = 45.00
Iteration 450: Best distance = 45.00

Final Best Route: [6, 7, 3, 2, 1, 4, 0, 5]
Final Best Distance: 45
